In [ ]:
import nltk

In [ ]:
import re

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from pprint import pprint

In [ ]:
import gensim

In [ ]:
import gensim.corpora as corpora


In [ ]:
from gensim.utils import simple_preprocess


In [ ]:
from gensim.models import CoherenceModel


In [ ]:
!pip install PYLDAvis


     |████████████████████████████████| 1.6MB 5.9MB/s 
  Created wheel for PYLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=a1fade4de4b3dee9f53e88c7b76ac2a968c2e0ca5edbed58e5e74d4d7e25d9b5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built PYLDAvis


In [ ]:
import pyLDAvis

In [ ]:
import pyLDAvis.gensim

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
%matplotlib inline


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')


In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')


In [ ]:
df

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [ ]:
df.shape

(11314, 3)

In [ ]:
df.head()['content'][0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [ ]:
all_text=df.content

In [ ]:
df['totalwords'] = df['content'].str.split().str.len()


In [ ]:
df.totalwords

0        123
1        123
2        339
3        113
4        171
        ... 
11309    337
11310    130
11311    139
11312    194
11313     75
Name: totalwords, Length: 11314, dtype: int64

In [ ]:
!python -m spacy download en_core_web_lg


     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180945 sha256=9a249de32dbc18ec60ea52799c5f186d0952715bb4613c019c8d1324c76f75cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-cfeluxhj/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
data = df.content.values.tolist()


In [ ]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]


<input>:1: DeprecationWarning:

invalid escape sequence \S

<input>:1: DeprecationWarning:

invalid escape sequence \S

<input>:1: DeprecationWarning:

invalid escape sequence \S

<ipython-input-34-9396b9abe3b2>:1: DeprecationWarning:

invalid escape sequence \S



In [ ]:
data = [re.sub('\s+', ' ', sent) for sent in data]


<input>:1: DeprecationWarning:

invalid escape sequence \s

<input>:1: DeprecationWarning:

invalid escape sequence \s

<input>:1: DeprecationWarning:

invalid escape sequence \s

<ipython-input-35-e591ccbe0189>:1: DeprecationWarning:

invalid escape sequence \s



In [ ]:
data = [re.sub("\'", "", sent) for sent in data]


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 

In [ ]:
print(bigram)

Phrases<971890 vocab, min_count=5, threshold=100, max_vocab_size=40000000>


In [ ]:
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning:

For a faster implementation, use the gensim.models.phrases.Phraser class



In [ ]:


bigram_mod = gensim.models.phrases.Phraser(bigram)


In [ ]:
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [ ]:
print(trigram_mod[bigram_mod[data_words[0]]])


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)


In [ ]:
data_words_bigrams = make_bigrams(data_words_nostops)


In [ ]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from spacy.lang.en import English
import spacy

In [ ]:

nlp = spacy.load('en', disable=['parser', 'ner'])


In [ ]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [ ]:
print(data_lemmatized[:1])


[['where', 'thing', 'subject', 'car', 'nntp_posting', 'host', 'rac_wam', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [ ]:
texts = data_lemmatized


In [ ]:
id2word = corpora.Dictionary(data_lemmatized)


In [ ]:
corpus = [id2word.doc2bow(text) for text in texts]


In [ ]:
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]]


In [ ]:
id2word[1]


'body'

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_posting', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('subject', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.046*"year" + 0.036*"game" + 0.025*"win" + 0.021*"play" + 0.019*"team" + '
  '0.018*"last" + 0.014*"run" + 0.014*"lose" + 0.012*"first" + 0.012*"hit"'),
 (1,
  '0.025*"use" + 0.024*"program" + 0.022*"file" + 0.017*"include" + '
  '0.015*"window" + 0.014*"available" + 0.013*"software" + 0.013*"information" '
  '+ 0.012*"source" + 0.011*"list"'),
 (2,
  '0.023*"bike" + 0.021*"light" + 0.020*"pin" + 0.018*"monitor" + '
  '0.016*"character" + 0.014*"normal" + 0.014*"ride" + 0.013*"drive" + '
  '0.012*"switch" + 0.011*"side"'),
 (3,
  '0.053*"good" + 0.052*"would" + 0.025*"make" + 0.019*"may" + 0.019*"much" + '
  '0.017*"lot" + 0.015*"well" + 0.011*"big" + 0.011*"pay" + 0.011*"money"'),
 (4,
  '0.025*"church" + 0.023*"christian" + 0.022*"slave" + 0.020*"faith" + '
  '0.019*"sin" + 0.017*"talk" + 0.016*"judge" + 0.015*"deny" + 0.014*"appear" '
  '+ 0.011*"heart"'),
 (5,
  '0.027*"car" + 0.021*"high" + 0.016*"price" + 0.016*"power" + 0.015*"cost" + '
  '0.014*"new" + 0.014*"low" + 0.

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


NameError: ignored

In [ ]:
from gensim.models.coherencemodel import CoherenceModel


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.44522161483195094


In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)



In [ ]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.301188  0.130372       1        1  21.164506
13    -0.248823 -0.123365       2        1  11.624432
16    -0.261437  0.110544       3        1  10.459088
7     -0.137676  0.017063       4        1   8.881553
1     -0.167173 -0.189846       5        1   8.194309
3     -0.173435  0.172027       6        1   7.367378
5     -0.040499 -0.126008       7        1   5.157488
0      0.009303  0.111955       8        1   3.618255
8     -0.022891 -0.268196       9        1   3.448523
6      0.077730 -0.088507      10        1   2.559403
2      0.111457  0.020941      11        1   2.392969
10     0.095764  0.037839      12        1   2.031908
9      0.119116  0.020057      13        1   1.929019
14     0.116036 -0.042521      14        1   1.874134
4      0.115214  0.055707      15        1   1.754159
12     0.121027  0.021482      16        1   1.621537
15     0.144921  0.020980      17        1   1.606057
11     0.138437  0.089818      18        1   1.543541
19     0.146769  0.005512      19        1   1.400083
17     0.157346  0.024146      20        1   1.371659, topic_info=               Term          Freq         Total Category  logprob  loglift
19             line  11523.000000  11523.000000  Default  30.0000  30.0000
176           would  12167.000000  12167.000000  Default  29.0000  29.0000
116            good   4562.000000   4562.000000  Default  28.0000  28.0000
199           write   9849.000000   9849.000000  Default  27.0000  27.0000
171             use   8645.000000   8645.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
11758   celebration     58.326384     59.289366  Topic20  -5.5934   4.2728
5159     prevention     54.330602     55.293585  Topic20  -5.6643   4.2716
2850   stuff_delete     54.023489     54.986471  Topic20  -5.6700   4.2715
1507    enforcement     52.377026     53.340008  Topic20  -5.7010   4.2709
1505           drug    399.411841    578.889556  Topic20  -3.6694   3.9180

[963 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
474      10  0.993459     absolute
4370     13  0.995118   absolutely
4640     19  0.995885  accelerator
1215      3  0.824274       accept
1215      4  0.175348       accept
...     ...       ...          ...
42        6  0.083863         year
42        7  0.096940         year
42        8  0.544970         year
723       8  0.990933    yesterday
4503     12  0.992781        youth

[1476 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 14, 17, 8, 2, 4, 6, 1, 9, 7, 3, 11, 10, 15, 5, 13, 16, 12, 20, 18])

In [ ]:
from gensim import corpora, models, similarities


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = F"/content/gdrive/My Drive/nlpProject/LdaModelFinal" 


In [ ]:
lda_model.save(path)


In [ ]:
model =  models.LdaModel.load(path)


In [ ]:
model.show_topics(200, 20)


[(0,
  '0.046*"year" + 0.036*"game" + 0.025*"win" + 0.021*"play" + 0.019*"team" + 0.018*"last" + 0.014*"run" + 0.014*"lose" + 0.012*"first" + 0.012*"hit" + 0.011*"watch" + 0.011*"score" + 0.010*"division" + 0.009*"wing" + 0.009*"next" + 0.008*"throw" + 0.008*"route" + 0.007*"bomb" + 0.007*"clearly" + 0.007*"average"'),
 (1,
  '0.025*"use" + 0.024*"program" + 0.022*"file" + 0.017*"include" + 0.015*"window" + 0.014*"available" + 0.013*"software" + 0.013*"information" + 0.012*"source" + 0.011*"list" + 0.011*"send" + 0.011*"also" + 0.010*"image" + 0.010*"copy" + 0.009*"entry" + 0.009*"support" + 0.009*"system" + 0.008*"application" + 0.008*"set" + 0.008*"user"'),
 (2,
  '0.023*"bike" + 0.021*"light" + 0.020*"pin" + 0.018*"monitor" + 0.016*"character" + 0.014*"normal" + 0.014*"ride" + 0.013*"drive" + 0.012*"switch" + 0.011*"side" + 0.010*"edge" + 0.009*"road" + 0.009*"motorcycle" + 0.008*"head" + 0.008*"btw" + 0.008*"turn" + 0.007*"foot" + 0.007*"will" + 0.007*"wave" + 0.006*"soon"'),
 (3,
